In [1]:
import sys, os
from pathlib import Path
import json
import time
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt

BASE_DIR = os.path.abspath(Path().resolve())
sys.path.append(os.path.join(BASE_DIR, "../"))
from ToppanUtils import *
from SiameseNet import *
import TemplateMatching as tm
from prefetch_generator import BackgroundGenerator
from keras import optimizers

Using TensorFlow backend.
/home/suzuki/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


image_dim_ordering:tf


In [7]:
import keras
keras.__version__

'2.0.8'

In [8]:
import tensorflow as tf
tf.__version__

'1.4.0'

### メダルデータ準備

In [10]:
### データが読み込めないーーーーーうわぁああああーーーーーっっっっっっ
data_base_path = "/data_server_storage/clients/kagadenshi/2019.06.05 メダル画像データ(検討用)/TAMA"
imageAs_path = data_base_path + "TAMA/"
imageBs_path = data_base_path + "MARU-1/"

imageAs = [
    cv2.imread(imageAs_path + img_name) for img_name in os.listdir(imageAs_path)
    if img_name != "Thumbs.db"
]
imageBs = [
    cv2.imread(imageBs_path + img_name) for img_name in os.listdir(imageBs_path)
    if img_name != "Thumbs.db"
]

FileNotFoundError: [Errno 2] No such file or directory: '/data_server_storage/clients/kagadenshi/2019.06.05 メダル画像データ(検討用)/TAMATAMA/'

### モデル準備

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

SIZE = 128
CHANNELS = 4
SEED = 1
LR = 1e-5
BATCH_SIZE = 128
EPOCHS = 300

In [3]:
### これを何回も実行してしまうと、レイヤーの名前が変わって、部分的なモデルの取り出しがうまくできなくなるかも。。
### モデルの定義の時に、すべてのレイヤーで name 指定をするべき。。
# adam = optimizers.Adam(lr=1e-4)
model = get_net(SIZE, CHANNELS)

In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
A (InputLayer)                   (None, 128, 128, 4)   0                                            
____________________________________________________________________________________________________
B (InputLayer)                   (None, 128, 128, 4)   0                                            
____________________________________________________________________________________________________
model_1 (Model)                  (None, 16, 16, 192)   195216      A[0][0]                          
                                                                   B[0][0]                          
____________________________________________________________________________________________________
model_2 (Model)                  (None, 1)             6259969     model_1[1][0]           

In [5]:
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes=True)

In [ ]:
### 部分的なモデルの取り出しとその出力を行うための関数　,,, だめぽ
# def get_layers(iunput_layer_name, output_layer_name, inputs):
#     layers = Model(inputs=model.get_layer(input_layer_name).input, outputs=model.get_layer(output_layer_name).output)
#     res = layers.predict(inputs)
#     return res

### メダルの登録

In [7]:
from keras.models import Model

### モデルの特徴量抽出部分を取り出す
input_layer_name = "A"
output_layer_name = "model_1"

# feature_extract_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)　-> できない
# feature_extract_model = Model(inputs=model.get_layer("A").input, outputs=model.get_layer(layer_name).output) -> できない

feature_extract_model = Model(inputs=model.get_layer(input_layer_name).input, outputs=model.get_layer(output_layer_name).get_output_at(1))

In [ ]:
### 取り出した特徴量抽出モデルで、登録したいメダルデータの特徴量抽出
feature_map_As = feature_extract_model.predict(image_As)

In [ ]:
### 登録したいメダルデータの特徴量を保存
# np.save("~.npz", feature_map_As) 適当

### 検査対象のメダルデータの特徴量抽出ならびに、その特徴量と登録したメダルの特徴量との距離を計算

In [ ]:
# model = load_model() 適当

In [15]:
# help(model)
# for i, layer in enumerate(model.layers):
#     if i == 0:
#         help(layer)
#     layer.set_weights(weights[i])

In [16]:
### モデルの特徴量抽出部分を取り出す
input_layer_name = "A" # B - 0だと上手くいかない
output_layer_name = "model_1"

feature_extract_model = Model(inputs=model.get_layer(input_layer_name).input, outputs=model.get_layer(output_layer_name).get_output_at(1))
# feature_map_Bs = feature_extract_model.predict(image_Bs)

### モデルの距離計算部分
# というか、モデルを２つに分割して１つのモデルとして上手く学習させることはできるのか、わからない。
distance_model = get_distance_model(size=SIZE)

# この時に学習した重みをどうにかして読み込まないといけないので、微妙？
layer_name = "model_2"
distance_model.set_weights(model.get_layer(layer_name).get_weights())


# 以下だめぽ　距離計算モデルを組み直して(インプットのプレースホルダーが無いとエラーになる)
# 距離計算部分を切り出して、１つのモデルとして利用
# その距離計算部分に対して、学習した重みを利用する方法へ変更してどうなるか

# ### 距離計算を２つに分けているが、ひとつでできる方がいいと思われる、、
# ### ２つの特徴量の距離計算 1
# input_layer_name = "subtract_1"
# output_layer_name = "max_pooling2d_4"

# midout_model_A = Model(inputs=model.get_layer(input_layer_name).input, outputs=model.get_layer(output_layer_name).output)
# # midout_As = midout_model_A.predict([feature_map_As, feature_map_Bs])

# input_layer_name = "subtract_2"
# output_layer_name = "max_pooling2d_5"

# midout_model_B = Model(inputs=model.get_layer(input_layer_name).input, outputs=model.get_layer(output_layer_name).output)
# # midout_Bs = midout_model_B.predict([feature_map_Bs, feature_map_As])

# ### ２つの特徴量の距離計算 2
# input_layer_name = "concatenate_2"
# output_layer_name = "dense_1"

# distance_model = Model(inputs=model.get_layer(input_layer_name).input, outputs=model.get_layer(output_layer_name).output)
# # res = distance_model.predict([midout_Bs, midout_As])

In [ ]:
### 登録したメダルデータの特徴量を読み込む
# feature_map_As = np.load("~.npz")　適当

In [ ]:
### 取り出したモデルたちでメダルデータの距離計算まで
feature_map_Bs = feature_extract_model.predict(image_Bs)
res = distance_model.predict([feature_map_As, feature_map_Bs])


# 以下は距離計算を２つに分けていた時のレガシー

# midout_As = midout_model_A.predict([feature_map_As, feature_map_Bs])
# midout_Bs = midout_model_B.predict([feature_map_Bs, feature_map_As])

# res = distance_model.predict([midout_Bs, midout_As])

### その他テスト

In [1]:
import keras

input1 = keras.layers.Input(shape=(5,))
# x1 = keras.layers.Dense(8, activation='relu')(input1)
input2 = keras.layers.Input(shape=(5,))
# x2 = keras.layers.Dense(8, activation='relu')(input2)
added = keras.layers.add([input1, input2])
# added = keras.layers.add([x1, x2])

# out = keras.layers.Dense(4)(added)
model = keras.models.Model(inputs=[input1, input2], outputs=added)

import numpy as np
a = np.array([[1,2,3,4,5]])
b = np.array([[2,3,4,5,6]])
pred = model.predict([a,b])

pred

Using TensorFlow backend.
/home/suzuki/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
